In [ ]:
# 

In [2]:
!pip install transformers==4.5.0 fugashi==1.1.0 ipadic==1.0.0
!pip install datasets

     |████████████████████████████████| 2.1 MB 5.4 MB/s 
     |████████████████████████████████| 486 kB 38.7 MB/s 
     |████████████████████████████████| 13.4 MB 31.1 MB/s 
     |████████████████████████████████| 3.3 MB 31.0 MB/s 
     |████████████████████████████████| 895 kB 42.8 MB/s 
  Created wheel for ipadic: filename=ipadic-1.0.0-py3-none-any.whl size=13556723 sha256=12a032b98bca945d3c71b2a7227985bbd43083ef962abb7060a7f95fbb940bfe
  Stored in directory: /root/.cache/pip/wheels/33/8b/99/cf0d27191876637cd3639a560f93aa982d7855ce826c94348b
Successfully built ipadic
     |████████████████████████████████| 325 kB 5.5 MB/s 
     |████████████████████████████████| 67 kB 4.2 MB/s 
     |████████████████████████████████| 212 kB 39.3 MB/s 
     |████████████████████████████████| 1.1 MB 40.8 MB/s 
     |████████████████████████████████| 134 kB 42.9 MB/s 
     |████████████████████████████████| 127 kB 45.6 MB/s 
     |████████████████████████████████| 271 kB 45.9 MB/s 
     |███████████████

In [3]:
# ライブラリ
import numpy as np
import pandas as pd
import torch
#from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
from transformers import BertJapaneseTokenizer, BertModel, BertConfig
from google.colab import drive

In [13]:
# パラメータ
MODEL_NAME = 'cl-tohoku/bert-base-japanese-whole-word-masking'

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

drive.mount("/content/drive/")
INPUT_PATH = "/content/drive/My Drive/NLP/work/10_my_task/01_bert_sentence_cls/input/" 
MODEL_PATH = "/content/drive/My Drive/NLP/work/10_my_task/01_bert_sentence_cls/output/model/" 
OUTPUT_PATH = "/content/drive/My Drive/NLP/work/10_my_task/01_bert_sentence_cls/output/" 

cuda
Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [14]:
# モジュールロード
tokenizer = BertJapaneseTokenizer.from_pretrained(MODEL_NAME) 

config = BertConfig.from_json_file(MODEL_PATH + '/last.bin/config.json')
model = BertModel(config).to(device)
model.load_state_dict(torch.load(MODEL_PATH + "/last.bin/pytorch_model.bin"))
# print(model)

<All keys matched successfully>

In [6]:
# データ
df = pd.read_csv(INPUT_PATH + "sample_3.csv", encoding='shift-jis')
print(df.shape)
df.head(3)

(20, 4)


,Q,T,Template,Train
0,振込限度額を教えてください。,振込限度額は100万円です。,1,1
1,１回の振込上限額はいくらですか？,振込限度額は100万円です。,1,0
2,口座開設したいです。,口座開設方法はこちらになります。,2,1


In [7]:
# データ選択
df = df[df['Train']==1].reset_index(drop=True)[['Q','T','Template']]
print(df.shape)
df.head(3)

(10, 3)


,Q,T,Template
0,振込限度額を教えてください。,振込限度額は100万円です。,1
1,口座開設したいです。,口座開設方法はこちらになります。,2
2,口座解約したいです。,口座解約の方法はこちらです。,3


In [9]:
# tokenizer
def tokenize_text(tokenizer, text):
  encoding = tokenizer(text, max_length=512, padding='max_length', truncation=True)
  encoding = {k: torch.tensor(v) for k, v in encoding.items()}
  return encoding

In [10]:
# encoding
encodings = []
for qt in (df['Q'] + "[SEP]" + df['T']).tolist():
  encoding = tokenize_text(tokenizer, qt)
  encodings.append(encoding)
print(len(encodings))

10


In [17]:
# ベクトル化
vec_qt = torch.tensor([]).to(device)
for batch in DataLoader(encodings, batch_size=4):
  input_ids = batch['input_ids'].to(device)
  attention_mask = batch['attention_mask'].to(device)

  with torch.no_grad():
    cls = model(input_ids, attention_mask, return_dict=True).last_hidden_state[:, 0, :] # CLSトークン
    #print(cls.shape)
    vec_qt = torch.cat((vec_qt, cls), 0)
print(vec_qt.shape)

torch.Size([4, 768])
torch.Size([4, 768])
torch.Size([2, 768])
torch.Size([10, 768])


In [18]:
# OUTPUT

# QTの文章ベクトル(学習用)
qt_ary = vec_qt.cpu().numpy()
np.save(OUTPUT_PATH + "vec_bert.npy" , qt_ary)
print(qt_ary.shape)

# 対応するテンプレート番号
df['Template'].to_csv(OUTPUT_PATH + "template.csv", index=False)
print(df['Template'].shape)

(10,)
